# 📚 RAG 시스템 완전 정복: 을GPT 0주차 

## 🎯 학습 목표
이 노트북에서는 다음과 같은 내용을 학습합니다:
- RAG 시스템의 기본 구조와 작동 원리
- 문서 로딩, 분할, 임베딩, 벡터 저장소 생성 과정
- LangChain을 활용한 질의응답 시스템 구축
- RAG 시스템 성능 개선 방법들

## 🛠️ 사용 기술 스택
- **LangChain**: RAG 파이프라인 구축 프레임워크
- **Ollama**: 로컬 LLM 및 임베딩 모델 실행
- **Chroma**: 벡터 데이터베이스
- **을지대학교 학업성적처리규정**: 실제 문서 데이터

## 📋 진행 순서
1. **기본 RAG 시스템 구축** (1단계~7단계)
2. **시스템 성능 분석 및 개선점 파악**
3. **다양한 개선 기법 적용 및 비교**
4. **최종 최적화된 시스템 테스트**

---

In [1]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain.chains import RetrievalQA

# # 1. 문서 불러오기 (지금 있는 파일)
# loader = DirectoryLoader(
#     path="data",              # 이 폴더 안의
#     glob="**/*.txt",                      # 모든 .txt 파일
#     loader_cls=TextLoader,                # 파일 하나당 TextLoader 사용
#     loader_kwargs={"encoding": "utf-8"}
# )

loader = TextLoader("data/을지대_학업성적처리규정.txt", encoding="utf-8")

documents = loader.load()

# 🔧 1단계: 필요한 라이브러리 Import 및 문서 로딩

## 📥 문서 로딩이란?
RAG 시스템의 첫 번째 단계는 **지식 베이스가 될 문서를 불러오는 것**입니다. 

### 사용하는 주요 컴포넌트:
- **DirectoryLoader**: 폴더 전체의 파일들을 한 번에 로딩
- **TextLoader**: 개별 텍스트 파일을 로딩
- **RecursiveCharacterTextSplitter**: 긴 문서를 작은 단위로 분할

### 💡 왜 문서를 불러와야 할까요?
- LLM은 훈련 데이터에 없는 최신 정보나 특정 도메인 지식을 모릅니다
- 우리의 문서(을지대학교 학업성적처리규정)를 LLM이 참조할 수 있도록 해야 합니다

---

In [2]:
# 2. 문서 쪼개기
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)

# ✂️ 2단계: 문서 분할 (Text Splitting)

## 🤔 왜 문서를 분할해야 할까요?
- **LLM의 컨텍스트 윈도우 제한**: 한 번에 처리할 수 있는 텍스트 길이가 제한됨
- **검색 정확도 향상**: 작은 단위로 나누면 더 정확한 정보 검색 가능
- **관련성 높은 정보 제공**: 질문과 관련된 특정 부분만 추출 가능

## ⚙️ 설정 파라미터 설명:
- **chunk_size=500**: 각 청크의 최대 글자 수
- **chunk_overlap=50**: 청크 간 겹치는 부분 (문맥 연결을 위해 필요)

### 💡 청크 크기가 중요한 이유:
- 너무 크면: 불필요한 정보까지 포함되어 답변이 부정확해질 수 있음
- 너무 작으면: 문맥이 부족해져 의미가 불분명해질 수 있음

---

In [3]:
# 3. 임베딩 모델 (Ollama 사용)
# ollama pull bge-m3
embedding = OllamaEmbeddings(model="bge-m3")  

# 🔢 3단계: 임베딩 모델 설정

## 🎯 임베딩(Embedding)이란?
텍스트를 **고차원 벡터(숫자 배열)로 변환**하는 과정입니다.

### 🧠 임베딩의 원리:
- 의미가 비슷한 텍스트는 비슷한 벡터값을 가짐
- 컴퓨터가 텍스트의 '의미'를 수학적으로 이해할 수 있게 됨
- 예: "개"와 "강아지"는 비슷한 벡터값을 가짐

## 🛠️ bge-m3 모델이란?
- **BGE**: Beijing Academy of Artificial Intelligence에서 개발
- **M3**: Multi-lingual, Multi-functionality, Multi-granularity
- **특징**: 다국어 지원, 한국어 성능 우수

### 💻 Ollama 사용 이유:
- 로컬에서 모델 실행 (인터넷 연결 불필요)
- 개인정보 보호 (데이터가 외부로 전송되지 않음)
- 비용 절약 (API 사용료 없음)

---

In [4]:
# 4. Chroma 벡터스토어에 저장
vectordb = Chroma.from_documents(
    docs,
    embedding=embedding,
    persist_directory="example_code/chroma_grade_rules"
)
vectordb.persist()

C:\Users\geon1\AppData\Local\Temp\ipykernel_15240\3149307985.py:7: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


# 🗄️ 4단계: 벡터 데이터베이스 생성

## 📊 벡터 데이터베이스란?
임베딩된 벡터들을 저장하고 **유사도 검색**을 빠르게 수행할 수 있는 특별한 데이터베이스입니다.

### 🔍 유사도 검색 과정:
1. 사용자 질문을 벡터로 변환
2. 저장된 문서 벡터들과 비교
3. 가장 유사한 벡터들을 찾아 반환

## 🌟 Chroma 데이터베이스 특징:
- **오픈소스**: 무료로 사용 가능
- **간단한 설정**: 복잡한 설치 과정 불필요
- **로컬 저장**: 데이터가 로컬에 안전하게 보관
- **빠른 검색**: 효율적인 벡터 유사도 검색

### 💾 persist_directory 설정:
- 벡터 데이터베이스를 디스크에 영구 저장
- 프로그램을 재시작해도 데이터가 유지됨
- 재사용 가능한 지식 베이스 구축

---

In [5]:
# 5. LLM (너의 모델)
# ollama run joonoh/HyperCLOVAX-SEED-Text-Instruct-1.5B
llm = ChatOllama(model="joonoh/HyperCLOVAX-SEED-Text-Instruct-1.5B")

# 🤖 5단계: LLM (Large Language Model) 설정

## 🧩 LLM의 역할
RAG 시스템에서 LLM은 **검색된 문서 정보를 바탕으로 자연스러운 답변을 생성**하는 역할을 합니다.

### 🔄 RAG에서 LLM 처리 과정:
1. 벡터 데이터베이스에서 관련 문서 검색
2. 검색된 문서와 사용자 질문을 함께 LLM에 전달
3. LLM이 문서 내용을 참고하여 정확한 답변 생성

## 🏠 HyperCLOVAX-SEED 모델 소개:
- **네이버에서 개발**한 한국어 특화 모델
- **1.5B 파라미터**: 적당한 크기로 빠른 응답 가능
- **Instruct 버전**: 지시사항을 잘 이해하고 따름
- **한국어 성능**: 한국어 문서 이해에 최적화

### ⚡ 로컬 실행의 장점:
- **빠른 응답속도** (네트워크 지연 없음)
- **무제한 사용** (API 호출 제한 없음)
- **프라이버시 보호** (데이터가 외부로 전송되지 않음)

---

In [6]:
# 6. RetrievalQA 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)

# 🔗 6단계: RAG 체인 구성

## ⛓️ RAG 체인이란?
여러 구성요소를 **순서대로 연결**하여 질문부터 답변까지 자동화된 파이프라인을 만드는 것입니다.

### 🔄 RAG 체인의 동작 순서:
1. **사용자 질문 입력**
2. **질문을 벡터로 변환** (임베딩)
3. **벡터 데이터베이스에서 유사한 문서 검색** (k=3개)
4. **검색된 문서 + 질문을 LLM에 전달**
5. **LLM이 문서를 참고하여 답변 생성**
6. **최종 답변 반환**

## ⚙️ RetrievalQA 설정 옵션:
- **chain_type**: 검색된 문서를 어떻게 처리할지 결정
- **retriever**: 벡터 데이터베이스에서 검색하는 방법
- **k=3**: 가장 유사한 문서 3개를 검색
- **return_source_documents=True**: 참고한 문서도 함께 반환

### 💡 왜 k=3일까요?
- 너무 적으면 (k=1): 정보가 부족할 수 있음
- 너무 많으면 (k=10): 관련 없는 정보까지 포함되어 혼란
- k=3: 적당한 균형점

---

In [7]:
# 7. 예시 질의
query = "재수강은 최대 몇 학점까지 가능한가요?"
result = qa_chain.invoke(query)

# 8. 출력
print("📌 답변:")
print(result["result"])

print("\n📚 참고 문서 (요약):")
for doc in result["source_documents"]:
    print("-", doc.page_content.strip().split("\n")[0])

📌 답변:
재수강은 최대 총 24학점까지 가능합니다. 단, 한 학기당 2과목으로 제한됩니다. (단, 2017학년도 신입생부터는 재학연한 이내 총 24학점까지 신청가능하며, 한 학기당 2과목으로 제한합니다.)

📚 참고 문서 (요약):
- ② 편입학생에 대하여는 전적 대학에서 이수한 교과목 및 학점을 심사하여 본 대학에
- ② 편입학생에 대하여는 전적 대학에서 이수한 교과목 및 학점을 심사하여 본 대학에
- ② 편입학생에 대하여는 전적 대학에서 이수한 교과목 및 학점을 심사하여 본 대학에


# 🧪 7단계: 기본 RAG 시스템 테스트

## 🎯 첫 번째 테스트
이제 구축한 RAG 시스템이 실제로 작동하는지 확인해보겠습니다!

### 📝 테스트 질문 분석:
**"재수강은 최대 몇 학점까지 가능한가요?"**
- 이 질문은 을지대학교 학업성적처리규정에서 구체적인 숫자 정보를 찾아야 합니다
- 시스템이 올바른 조항을 찾고 정확한 학점 수를 답할 수 있는지 확인

### 🔍 결과 분석 포인트:
1. **답변의 정확성**: 규정에 명시된 정확한 학점 수가 나오는가?
2. **근거 문서**: 어떤 문서 부분을 참고했는가?
3. **답변의 자연스러움**: 사람이 이해하기 쉬운 형태인가?

### 📊 output 구조:
- **result**: LLM이 생성한 최종 답변
- **source_documents**: 답변 생성에 사용된 문서 조각들

---

# RAG 시스템 개선 방법들

답변 정확도를 개선하기 위한 여러 방법들을 적용해보겠습니다:

1. **더 나은 텍스트 분할**: 의미 단위로 분할
2. **개선된 검색**: 유사도 점수 기반 필터링
3. **더 나은 프롬프트**: 컨텍스트를 활용한 구체적인 지시
4. **하이브리드 검색**: 키워드 + 의미 검색 결합
5. **답변 검증**: 소스 문서와의 일치성 확인

In [8]:
## 개선 방법 1: 더 나은 텍스트 분할

from langchain.text_splitter import CharacterTextSplitter
import re

def smart_text_splitter(documents):
    """의미 단위로 문서를 분할하는 개선된 함수"""
    # 한국어 문서에 맞는 구분자 설정
    korean_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,  # 더 작은 청크 크기
        chunk_overlap=100,  # 더 큰 오버랩
        separators=[
            "\n\n",  # 단락 구분
            "\n",    # 줄바꿈
            "。",     # 마침표
            ".",
            " ",     # 공백
            ""
        ],
        keep_separator=True
    )
    
    return korean_splitter.split_documents(documents)

# 개선된 분할 적용
improved_docs = smart_text_splitter(documents)
print(f"기존 문서 수: {len(docs)}")
print(f"개선된 문서 수: {len(improved_docs)}")
print(f"\n첫 번째 청크 예시:")
print(improved_docs[0].page_content[:200] + "...")

기존 문서 수: 10
개선된 문서 수: 20

첫 번째 청크 예시:
학업성적처리규정
제정 2007. 3. 1.
개정 2015. 3. 1.
개정 2017. 3. 1.
개정 2017. 9. 1.
개정 2018. 9. 1.
개정 2020. 3. 1.
개정 2022. 2. 1.
개정 2023. 3. 1.
개정 2024. 3. 1.
개정 2024. 4. 1.
개정 2025. 1.15.
제1조(목적) 이 규정은 을지대학교(이하 “본교...


# 🚀 RAG 시스템 성능 개선 시작!

이제 기본 RAG 시스템의 답변을 확인했으니, **더 정확하고 유용한 답변**을 위해 시스템을 개선해보겠습니다.

## 🔧 개선 방법 1: 스마트한 텍스트 분할

### 🤔 기존 방식의 문제점:
- 단순히 글자 수로만 분할
- 문장이나 단락의 의미 단위를 고려하지 않음
- 중요한 정보가 여러 청크로 나뉠 수 있음

### ✨ 개선된 분할 전략:
1. **더 작은 청크 크기** (500→300): 더 정확한 검색
2. **더 큰 오버랩** (50→100): 문맥 연결성 향상
3. **한국어 맞춤 구분자**: 단락, 문장 단위로 자연스럽게 분할

### 📊 구분자 우선순위:
1. `\n\n` - 단락 구분 (가장 자연스러운 분할점)
2. `\n` - 줄바꿈
3. `。`, `.` - 문장 끝
4. ` ` - 공백 (최후의 수단)

---

In [9]:
## 개선 방법 2: 새로운 벡터스토어 구성

# 개선된 문서로 새로운 벡터스토어 생성
improved_vectordb = Chroma.from_documents(
    improved_docs,
    embedding=embedding,
    persist_directory="example_code/chroma_grade_rules_improved"
)
improved_vectordb.persist()

print("개선된 벡터스토어가 생성되었습니다.")

개선된 벡터스토어가 생성되었습니다.


# 🔧 개선 방법 2: 새로운 벡터스토어 구성

## 🆕 왜 새로운 벡터스토어를 만들까요?
개선된 텍스트 분할 방식으로 나뉜 문서들을 저장하기 위해 **별도의 벡터스토어**를 생성합니다.

### 📂 저장 경로 구분:
- **기존**: `chroma_grade_rules` - 원본 분할 방식
- **개선**: `chroma_grade_rules_improved` - 개선된 분할 방식

### 💡 분리 저장의 이점:
1. **성능 비교 가능**: 원본과 개선 버전 직접 비교
2. **롤백 가능**: 문제시 원본 버전으로 복구
3. **실험 환경**: 다양한 설정 테스트 가능

### ⚡ 벡터화 과정:
1. 개선된 문서 청크들을 임베딩 모델로 벡터화
2. Chroma 데이터베이스에 저장
3. 디스크에 영구 저장 (persist)

---

In [10]:
## 개선 방법 3: 커스텀 프롬프트 템플릿

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# 더 구체적인 프롬프트 템플릿 생성
custom_prompt = PromptTemplate(
    template="""당신은 을지대학교 학업성적처리규정 전문가입니다. 주어진 문서를 바탕으로 정확하고 구체적인 답변을 제공해주세요.

문서 내용:
{context}

질문: {question}

답변 가이드라인:
1. 문서에 명시된 정확한 정보만 사용하세요
2. 학점, 기간, 조건 등 숫자 정보는 정확히 인용하세요
3. 관련 조항이나 예외사항이 있다면 함께 언급하세요
4. 확실하지 않은 정보는 "문서에서 명확히 확인할 수 없습니다"라고 말하세요

답변:""",
    input_variables=["context", "question"]
)

# 검색 결과 포맷팅 함수
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 개선된 RAG 체인 생성
improved_rag_chain = (
    {"context": improved_vectordb.as_retriever(search_kwargs={"k": 5}) | format_docs, 
     "question": RunnablePassthrough()}
    | custom_prompt
    | llm
    | StrOutputParser()
)

print("커스텀 프롬프트 템플릿이 적용된 RAG 체인이 생성되었습니다.")

커스텀 프롬프트 템플릿이 적용된 RAG 체인이 생성되었습니다.


# 🔧 개선 방법 3: 커스텀 프롬프트 템플릿

## 📝 프롬프트 엔지니어링의 중요성
프롬프트는 LLM에게 **"어떻게 답변해야 하는지"** 알려주는 지시서입니다. 좋은 프롬프트는 답변 품질을 크게 향상시킵니다.

### 🆚 기본 vs 커스텀 프롬프트:

**🔸 기본 RetrievalQA 프롬프트:**
- 단순히 문서와 질문만 전달
- 구체적인 답변 형식 지정 없음
- 출처나 근거 명시 요구 없음

**🔸 커스텀 프롬프트 개선점:**
1. **명확한 역할 정의**: "을지대학교 학업성적처리규정 전문가"
2. **구체적인 답변 가이드라인**: 정확한 정보만 사용, 숫자 정확히 인용
3. **안전장치**: 불확실한 정보는 명시하도록 지시

### 🎯 프롬프트 구성 요소:
- **컨텍스트 영역**: 검색된 문서 내용
- **질문 영역**: 사용자의 실제 질문
- **지시사항**: 답변 방식과 주의사항

### 🔄 새로운 RAG 체인 구조:
기존의 단순한 RetrievalQA 대신 **RunnableSequence**를 사용하여 더 세밀한 제어가 가능합니다.

---

In [11]:
## 개선 방법 4: 유사도 점수 기반 필터링

def enhanced_retrieval(query, vectorstore, threshold=0.5, k=5):
    """유사도 점수를 기반으로 한 향상된 검색"""
    # similarity_search_with_score를 사용하여 점수와 함께 검색
    docs_with_scores = vectorstore.similarity_search_with_score(query, k=k*2)
    
    # 임계값보다 높은 점수의 문서만 필터링
    filtered_docs = [doc for doc, score in docs_with_scores if score > threshold]
    
    # 상위 k개 문서만 반환
    return filtered_docs[:k]

# 테스트를 위한 함수
def test_enhanced_qa(query, vectorstore, custom_prompt, llm):
    """개선된 QA 시스템 테스트"""
    # 향상된 검색 수행
    relevant_docs = enhanced_retrieval(query, vectorstore)
    
    if not relevant_docs:
        return "관련 문서를 찾을 수 없습니다. 질문을 다시 확인해주세요."
    
    # 문서 내용 포맷팅
    context = "\n\n".join([doc.page_content for doc in relevant_docs])
    
    # 프롬프트에 입력
    prompt_input = custom_prompt.format(context=context, question=query)
    
    # LLM 호출
    response = llm.invoke(prompt_input)
    
    return response.content if hasattr(response, 'content') else str(response)

print("향상된 검색 함수가 준비되었습니다.")

향상된 검색 함수가 준비되었습니다.


# 🔧 개선 방법 4: 유사도 점수 기반 필터링

## 📊 유사도 점수란?
벡터 간의 **거리(유사성)**을 0~1 사이의 숫자로 나타낸 것입니다.
- **점수가 낮을수록** = 더 유사함
- **점수가 높을수록** = 덜 유사함

### 🤔 왜 필터링이 필요할까요?
기본 RAG는 **무조건 k개의 문서**를 가져옵니다. 하지만:
- 질문과 관련성이 낮은 문서도 포함될 수 있음
- 관련성 낮은 문서가 답변을 혼란스럽게 만들 수 있음

### ✨ 개선된 검색 전략:
1. **더 많이 검색** (k×2): 후보군 확장
2. **점수로 필터링**: 임계값보다 좋은 문서만 선택
3. **상위 k개만 선택**: 최종적으로 필요한 개수만 사용

### 🎛️ 임계값(threshold) 설정:
- **0.5**: 보통 수준 (너무 관련 없는 문서 제외)
- **0.3**: 엄격한 수준 (매우 관련 있는 문서만)
- **0.8**: 관대한 수준 (웬만한 문서 모두 포함)

### 🧪 실험적 접근:
다양한 임계값을 테스트해서 최적의 값을 찾아보겠습니다!

---

In [12]:
## 개선 방법 5: 실제 테스트 및 비교

# 기존 시스템과 개선된 시스템 비교
test_queries = [
    "재수강은 최대 몇 학점까지 가능한가요?",
    "성적 경고는 언제 받나요?",
    "학점 취소는 몇 과목까지 가능한가요?",
    "졸업 평점은 어떻게 계산하나요?"
]

def compare_systems(query):
    """기존 시스템과 개선된 시스템 비교"""
    print(f"🔍 질문: {query}")
    print("="*50)
    
    # 기존 시스템
    original_result = qa_chain.invoke(query)
    print("📋 기존 시스템 답변:")
    print(original_result["result"])
    print()
    
    # 개선된 시스템
    improved_answer = improved_rag_chain.invoke(query)
    print("✨ 개선된 시스템 답변:")
    print(improved_answer)
    print("\n" + "="*50 + "\n")

# 첫 번째 질문으로 테스트
compare_systems(test_queries[0])

🔍 질문: 재수강은 최대 몇 학점까지 가능한가요?
📋 기존 시스템 답변:
재수강은 총 24학점까지 가능합니다. 단, 한 학기당 2과목으로 제한됩니다. (2017학년도 신입생부터는 재학연한 이내)

✨ 개선된 시스템 답변:
문서에 따르면 한 학기에 학점취소를 위한 재수강신청은 최대 6학점까지 가능합니다. 이는 단, 2017학년도 신입생부터는 재학연한 이내 총 24학점까지 신청가능하며, 한 학기당 2과목으로 제한된다는 예외사항이 있습니다.




# 🔧 개선 방법 5: 실제 테스트 및 성능 비교

## 🆚 A/B 테스트의 중요성
**기존 시스템**과 **개선된 시스템**을 직접 비교해서 실제로 성능이 향상되었는지 확인해야 합니다.

### 📋 비교 테스트 항목:
1. **답변 정확성**: 올바른 정보를 제공하는가?
2. **답변 완성도**: 필요한 정보가 빠짐없이 포함되었는가?
3. **답변 명확성**: 이해하기 쉽고 구체적인가?
4. **근거 제시**: 답변의 출처가 명확한가?

### 🎯 테스트 질문 설계:
다양한 유형의 질문으로 시스템의 범용성을 확인합니다:
- **수치 정보**: "최대 몇 학점까지?"
- **조건 정보**: "언제 받나요?"
- **절차 정보**: "어떻게 계산하나요?"

### 📊 결과 분석 포인트:
- 답변이 구체적인 숫자를 포함하는가?
- 관련 조항이나 예외사항을 언급하는가?
- 문서에 없는 정보를 추측하지 않는가?

---

# ⚡ 즉시 적용 가능한 세부 최적화

## 🔍 하이퍼파라미터 튜닝
머신러닝에서 **하이퍼파라미터**는 모델의 성능을 좌우하는 중요한 설정값들입니다. RAG 시스템에서도 최적의 값을 찾는 것이 중요합니다.

### 🎛️ 주요 튜닝 대상:

**1. 검색 문서 개수 (k값)**
- k=3: 빠르지만 정보 부족 가능
- k=5: 균형점 (일반적으로 권장)
- k=7: 풍부한 정보, 하지만 노이즈 증가 가능

**2. 유사도 임계값 (threshold)**
- 0.3: 매우 엄격 (높은 품질, 적은 문서)
- 0.5: 보통 수준 (균형)
- 0.8: 관대함 (많은 문서, 품질 다양)

### 🧪 실험적 최적화:
각 파라미터를 체계적으로 테스트하여 우리 데이터에 가장 적합한 설정을 찾아보겠습니다.

### 🎯 최종 프롬프트 개선:
더욱 구체적이고 명확한 지시사항으로 답변 품질을 극대화합니다.

---

In [14]:
# 개선된 프롬프트로 새로운 체인 생성
best_rag_chain = (
    {"context": improved_vectordb.as_retriever(search_kwargs={"k": 3}) | format_docs, 
     "question": RunnablePassthrough()}
    | better_prompt
    | llm
    | StrOutputParser()
)

# 최종 개선된 시스템으로 테스트
query = "재수강은 최대 몇 학점까지 가능한가요?"

print("🚀 최종 개선된 시스템 답변:")
print("="*50)
final_answer = best_rag_chain.invoke(query)
print(final_answer)
print("="*50)

# 검색된 문서의 유사도 점수도 확인
print("\n📊 검색된 문서 유사도 점수:")
docs_with_scores = improved_vectordb.similarity_search_with_score(query, k=3)
for i, (doc, score) in enumerate(docs_with_scores):
    print(f"{i+1}. 점수: {score:.3f}")
    print(f"   내용: {doc.page_content[:150]}...")
    print()

🚀 최종 개선된 시스템 답변:
규정에 따르면 한 학기에 학점취소를 위한 재수강신청은 6학점 이내에 한하여 허용됩니다. 예외사항으로, 2017학년도 신입생부터는 재학연한 이내 총 24학점까지 신청이 가능합니다. (개정 2017.9.1.)

📊 검색된 문서 유사도 점수:
1. 점수: 372.964
   내용: 3-3-14~2
서 요구되는 교과목 및 학점만 인정하고 정해진 잔여 과정은 수강을 신청하여 이수하도록 하여야 한다.
③ 정해진 학점미달로 졸업이 보류된 자는 학점 미취득 과목에 한하여 재수강을 신청하여야 한다.
④ 한 학기에 학점취소를 위한 재수강신청은 6학점 이내에 ...

2. 점수: 372.964
   내용: 3-3-14~2
서 요구되는 교과목 및 학점만 인정하고 정해진 잔여 과정은 수강을 신청하여 이수하도록 하여야 한다.
③ 정해진 학점미달로 졸업이 보류된 자는 학점 미취득 과목에 한하여 재수강을 신청하여야 한다.
④ 한 학기에 학점취소를 위한 재수강신청은 6학점 이내에 ...

3. 점수: 372.964
   내용: 3-3-14~2
서 요구되는 교과목 및 학점만 인정하고 정해진 잔여 과정은 수강을 신청하여 이수하도록 하여야 한다.
③ 정해진 학점미달로 졸업이 보류된 자는 학점 미취득 과목에 한하여 재수강을 신청하여야 한다.
④ 한 학기에 학점취소를 위한 재수강신청은 6학점 이내에 ...

